<a href="https://colab.research.google.com/github/dhorvath/Solar-Sentiment-Predictor/blob/main/Solar_Sentiment_Predictor_Natural_Language_into_Structured_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pandas numpy openai langchain langchain-openai pydantic

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 335.9/335.9 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.6/990.6 kB 29.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 379.9/379.9 kB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.1/140.1 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 34.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.7 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfu

## Imports & API Auth

In [11]:
import os
import pandas as pd
import re
import json
import openai

from openai import OpenAI
from google.colab import userdata
from langchain_openai import ChatOpenAI
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain.output_parsers import PydanticOutputParser
from langchain.prompts import ChatPromptTemplate
from typing import Literal, List, Optional
from pydantic import BaseModel, Field, conint, constr

# API
open_ai_key = userdata.get('open_ai_key')
client = OpenAI(api_key=open_ai_key)

os.environ["OPENAI_API_KEY"] = open_ai_key

In [12]:
# The temperature impacts the randomness of the output, which in this case we don't want any randomness so we define it as 0.0
temperature = 0.0
model = "gpt-4"

llm = ChatOpenAI(model=model, temperature=temperature)

## Load CSV into a DataFrame

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Load the CSV file into a DataFrame
csv_file_path = '/content/drive/MyDrive/Career/Build Something!/Community Solar Sentiment/Renewable Rejection Database_RAW - RAW.csv'
df = pd.read_csv(csv_file_path)

# Display the DataFrame to understand its structure
df.head()

,Notes
0,"05/30/2024, VA, Augusta County, Planning Commi..."
1,"05/11/2024, CO, Montrose County, Board of Coun..."
2,"05/07/2024, MA, Town of Northfield, Annual Tow..."
3,"05/07/2024, NE, Scotts Bluff County, County Bo..."
4,"04/26/2024, CO, San Miguel County, Board of Co..."


## Define the schema

In [ ]:
class SearchSchema(BaseModel):
    year: conint(ge=1900, le=2100) = Field(description="the year the project was initially proposed in YYYY format")
    name: str = Field(description="name of the proposed project")
    state: Literal[
        'AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'FL', 'GA', 'HI', 'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'LA', 'ME', 'MD',
        'MA', 'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ', 'NM', 'NY', 'NC', 'ND', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC',
        'SD', 'TN', 'TX', 'UT', 'VT', 'VA', 'WA', 'WV', 'WI', 'WY'
    ] = Field(description="the state code for the location")
    jurisdiction: str = Field(description="city, township, county, or local jurisdiction the renewable project is located in")
    status: Literal["Stopped", "Ongoing: appeals, lawsuits, etc.", "Ongoing: further review", "Approved/completed", "Unknown"] = Field(description="Status of the project")
    type: Literal["solar", "wind"] = Field(description="the type of renewable energy project")
    size_in_MW: Optional[int] = Field(description="size of the solar or wind project in terms of MW or megawatts.")
    size_in_acres: int = Field(description="size of the solar or wind project in terms of acres or the physical area the project is sited on. If there are multiple projects included summarize the total number of acres mentioned.")
    concerns: List[Literal[
        'impact: visual and landscape',
        'impact: removing land from agricultural production (e.g. "food vs fuel")',
        'impact: public health and safety',
        'impact: environmental impacts and wildlife',
        'impact: tribal rights',
        'impact: property values',
        'impact: water use',
        'impact: other',
        'process: limited information provided to community',
        'process: unfair process',
        'process: lack of community influence on design'
        'unknown'
    ]] = Field(description="the reason the project was opposed by the community")
    opposition_method: Literal["Comments at public hearings", "Letter-writing campaigns", "Petitions", "Participation in admin proceedings", "Lawsuits"] = Field(description="The method by which the community opposed the project")
    opposition_group: str = Field(description="Name of the group or groups opposing the renewable project")
    farmland: Optional[Literal["true", "false","unknown"]] = Field(description="whether the project is on farmland (true, false, unknown)")
    developer: Optional[str] = Field(description="name of developer or company which is trying to build the renewable projects")

# Example usage
example = SearchSchema(
    year=2023,
    name="Sunshine Solar Farm",
    state="CA",
    jurisdiction="Los Angeles County",
    status="Stopped",
    type="solar",
    size_in_MW=150,
    size_in_acres=70,
    concerns=[
        'impact: visual and landscape',
        'impact: environmental impacts and wildlife',
        'process: limited information provided to community'
    ],
    opposition_method="Comments at public hearings",
    opposition_group="Save Our Lands",
    farmland="true",
    developer="Green Energy Co."
)

print(example)

year=2023 name='Sunshine Solar Farm' state='CA' jurisdiction='Los Angeles County' status='Stopped' type='solar' size_in_MW=150 size_in_acres=70 concerns=['impact: visual and landscape', 'impact: environmental impacts and wildlife', 'process: limited information provided to community'] opposition_method='Comments at public hearings' opposition_group='Save Our Lands' farmland='true' developer='Green Energy Co.'


## Create the Pydantic output parser

In [ ]:
# Create the Pydantic output parser
pydantic_parser = PydanticOutputParser(pydantic_object=SearchSchema)

# Generate the format instructions
format_instructions = pydantic_parser.get_format_instructions()

# Print the format instructions
print(format_instructions)

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"year": {"description": "the year the project was initially proposed in YYYY format", "maximum": 2100, "minimum": 1900, "title": "Year", "type": "integer"}, "name": {"description": "name of the proposed project", "title": "Name", "type": "string"}, "state": {"description": "the state code for the location", "enum": ["AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DE", "FL", "GA", "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD", "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ", "NM", "NY", "NC", "ND", "OH", "OK",

## Define the prompt template

In [ ]:
PROJECT_DETAILS = """
Your task is to parse the details of the renewable project(s) from the provided Notes. Summarize all relevant project details. For both the size_in_MW and size_in_acres schema fields, if there are multiple projects mentioned in Notes, summarize the total number of megawatts(MW) and/or acres.

{format_instructions}

Recipe Search Request:
{request}
"""

prompt = ChatPromptTemplate.from_template(
    template=PROJECT_DETAILS,
    partial_variables = {
        "format_instructions": format_instructions # passing in the formatting instructions created earlier in place of "format_instructions" placeholder
    }
)

## Define the full chain

In [ ]:
full_chain = {"request": lambda x: x["request"]} | prompt | llm

## Process each row

In [ ]:
# Function to process each note
def process_notes(note):
    request = note
    result = full_chain.invoke({"request": request})
    return result.content

# Iterate over each row in the DataFrame and process the "Notes" column
results = []
original_notes = []
for index, row in df.iterrows():
    note = row['Notes']
    result = process_notes(note)

    # Only print the results for the first 5 rows
    if index < 5:
        print(f"Result for row {index}: {result}")
    results.append(result)
    original_notes.append(note)

Result for row 0: {
  "year": 2024,
  "name": "AES Solar Project",
  "state": "VA",
  "jurisdiction": "Augusta County",
  "status": "Stopped",
  "type": "solar",
  "size_in_MW": null,
  "size_in_acres": 1600,
  "concerns": [
    "impact: visual and landscape",
    "impact: property values",
    "impact: other"
  ],
  "opposition_method": "Comments at public hearings",
  "opposition_group": "Augusta County Planning Commission",
  "farmland": "unknown",
  "developer": "AES"
}
Result for row 1: {"year": 2024, "name": "Unnamed Solar Project", "state": "CO", "jurisdiction": "Montrose County", "status": "Ongoing: further review", "type": "solar", "size_in_MW": null, "size_in_acres": null, "concerns": ["process: unfair process", "impact: removing land from agricultural production (e.g. \"food vs fuel\")"], "opposition_method": "Comments at public hearings", "opposition_group": "Board of County Commissioners", "farmland": "true", "developer": "Enel Energy"}
Result for row 2: {"year": 2024, "na

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

## Function to extract and parse JSON and create new DataFrame

In [ ]:
# Function to extract and parse JSON and parsing explanation from the result strings
def extract_data_from_result(result_str, original_note):
    try:
        # Extract the JSON part from the string
        json_str = re.search(r'\{.*\}', result_str, re.DOTALL).group(0)
        parsed_json = json.loads(json_str)

        # Extract the parsing explanation
        explanation = result_str.split(json_str)[-1].strip()

        return {
            "Notes": original_note,
            "year": parsed_json.get("year", 0),
            "name": parsed_json.get("name", ""),
            "state": parsed_json.get("state", ""),
            "jurisdiction": parsed_json.get("jurisdiction", ""),
            "status": parsed_json.get("status", ""),
            "type": parsed_json.get("type", ""),
            "size_in_MW": parsed_json.get("size_in_MW", 0),
            "size_in_acres": parsed_json.get("size_in_acres", 0),
            "concerns": parsed_json.get("concerns", []),
            "opposition_method": parsed_json.get("opposition_method", ""),
            "opposition_group": parsed_json.get("opposition_group", ""),
            "farmland": parsed_json.get("farmland", ""),
            "developer": parsed_json.get("developer", ""),
            "parsing_explanation": explanation
        }
    except (json.JSONDecodeError, AttributeError) as e:
        print(f"Invalid JSON: {result_str}\nError: {e}")
        return None

# Iterate over each row in the DataFrame, process the "Notes" column, and extract data
results = [process_notes(note) for note in df['Notes']]
extracted_data = [extract_data_from_result(result, note) for result, note in zip(results, df['Notes'])]

# Filter out any None entries due to invalid JSON
cleaned_data = [data for data in extracted_data if data is not None]

# Create a new DataFrame from the cleaned data
df_cleaned = pd.DataFrame(cleaned_data)

## Save the new DataFrame to a CSV file

In [ ]:
# Save the new DataFrame to a CSV file
output_path = '/content/drive/MyDrive/Career/Build Something!/Community Solar Sentiment/parsed_results_8.csv'
df_cleaned.to_csv(output_path, index=False)

# Display the DataFrame
print(df_cleaned.head())

## Single request (testing purposes)

In [ ]:
def test_single_note_json_formatted(note):
    # Process the single note
    result = process_notes(note)

    # Extract data using the previously defined function
    extracted_data = extract_data_from_result(result, note)

    # Print the result in JSON format
    if extracted_data:
        print(json.dumps(extracted_data, indent=2, default=str))
    else:
        print("Failed to parse the note. Check the format and schema compatibility.")

# Example Note
test_note = "Aramis and SunWalker Solar Projects: The Aramis (410 acres) and SunWalker (70 acres) solar projects, near Livermore, have been met with opposition by local politicians and interest groups. Two residents running in the election for the Alameda County Board of Supervisors, in partnership with citizen group Save North Livermore Valley, urged the board to place a moratorium on solar development on agricultural land. Opponents of the projects argue that the project's locations “conflict with agriculture, natural habitat, open space, and visual and scenic resources.” After the East County Board of Zoning Adjustments approved both projects, local groups have stated their intent to appeal the decision. As of December 2020, four separate appeals have been filed."

# Testing the function with the example note
test_single_note_json_formatted(test_note)